In [22]:
import pandas as pd
import csv
import glob

In [24]:
# To preview data 

row_count = 5
chunks = []
for chunk in pd.read_csv('./data/2018_public_lar_csv.csv', chunksize=row_count): 
    print chunk.head()
    # yeschunk = ['derived_msa_md']=='41180'

   activity_year                   lei  derived_msa_md state_code  \
0           2018  549300HW662MN1WU8550           40140         CA   
1           2018  549300HW662MN1WU8550           33460         MN   
2           2018  549300HW662MN1WU8550           47664         MI   
3           2018  549300HW662MN1WU8550           12060         GA   
4           2018  549300HW662MN1WU8550           29820         NV   

   county_code  census_tract conforming_loan_limit derived_loan_product_type  \
0         6071    6071009709                     C            FHA:First Lien   
1        27141   27141030101                     C   Conventional:First Lien   
2        26125   26125183900                     C   Conventional:First Lien   
3        13057   13057090400                     C   Conventional:First Lien   
4        32003   32003005853                     C   Conventional:First Lien   

              derived_dwelling_category       derived_ethnicity  ...  \
0  Single Family (1-4 Units):Sit

    activity_year                   lei  derived_msa_md state_code  \
20           2018  549300HW662MN1WU8550           27340         NC   
21           2018  549300HW662MN1WU8550           47664         MI   
22           2018  549300HW662MN1WU8550           47664         MI   
23           2018  549300HW662MN1WU8550           16974         IL   
24           2018  549300HW662MN1WU8550           40140         CA   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
20        37133   37133002201                     C            FHA:First Lien   
21        26125   26125128900                     C   Conventional:First Lien   
22        26125   26125145400                     C   Conventional:First Lien   
23        17031   17031808002                     C   Conventional:First Lien   
24         6065    6065046403                     C   Conventional:First Lien   

               derived_dwelling_category       derived_ethnicity  ...  \
20  Single Family (

    activity_year                   lei  derived_msa_md state_code  \
40           2018  549300HW662MN1WU8550           29180         LA   
41           2018  549300HW662MN1WU8550           48424         FL   
42           2018  549300HW662MN1WU8550           27260         FL   
43           2018  549300HW662MN1WU8550           29460         FL   
44           2018  549300HW662MN1WU8550           31084         CA   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
40        22055   22055002103                     C            FHA:First Lien   
41        12099   12099005814                     C            FHA:First Lien   
42        12109   12109020804                     C   Conventional:First Lien   
43        12105   12105011902                     C   Conventional:First Lien   
44         6037    6037236100                     C   Conventional:First Lien   

               derived_dwelling_category        derived_ethnicity  ...  \
40  Single Family 

    activity_year                   lei  derived_msa_md state_code  \
60           2018  549300HW662MN1WU8550           35614         NJ   
61           2018  549300HW662MN1WU8550           11244         CA   
62           2018  549300HW662MN1WU8550           36740         FL   
63           2018  549300HW662MN1WU8550           38300         PA   
64           2018  549300HW662MN1WU8550           36740         FL   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
60        34017   34017006100                     C   Conventional:First Lien   
61         6059    6059063903                     C   Conventional:First Lien   
62        12097   12097043204                     C             VA:First Lien   
63        42003   42003413201                     C   Conventional:First Lien   
64        12095   12095017804                     C            FHA:First Lien   

               derived_dwelling_category       derived_ethnicity  ...  \
60  Single Family (

    activity_year                   lei  derived_msa_md state_code  \
80           2018  549300HW662MN1WU8550           12580         MD   
81           2018  549300HW662MN1WU8550           31140         KY   
82           2018  549300HW662MN1WU8550           36260         UT   
83           2018  549300HW662MN1WU8550           30860         UT   
84           2018  549300HW662MN1WU8550           30860         UT   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
80        24005   24005400400                     C   Conventional:First Lien   
81        21111   21111011110                     C   Conventional:First Lien   
82        49011   49011126002                     C   Conventional:First Lien   
83        49005   49005000202                     C   Conventional:First Lien   
84        49005   49005000202                     C   Conventional:First Lien   

               derived_dwelling_category        derived_ethnicity  ...  \
80  Single Family 

...

     activity_year                   lei  derived_msa_md state_code  \
155           2018  549300HW662MN1WU8550           22744         FL   
156           2018  549300HW662MN1WU8550           36260         UT   
157           2018  549300HW662MN1WU8550           16974         IL   
158           2018  549300HW662MN1WU8550           19740         CO   
159           2018  549300HW662MN1WU8550           12060         GA   

     county_code  census_tract conforming_loan_limit  \
155        12011   12011043100                     C   
156        49011   49011126801                     C   
157        17031   17031081403                     C   
158         8005    8005007302                     C   
159        13067   13067030310                     C   

    derived_loan_product_type             derived_dwelling_category  \
155   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
156   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
157   Conventional:Firs

     activity_year                   lei  derived_msa_md state_code  \
170           2018  549300HW662MN1WU8550           10580         NY   
171           2018  549300HW662MN1WU8550           37964         PA   
172           2018  549300HW662MN1WU8550           24540         CO   
173           2018  549300HW662MN1WU8550           33124         FL   
174           2018  549300HW662MN1WU8550           11460         MI   

     county_code  census_tract conforming_loan_limit  \
170        36083   36083051600                     C   
171        42101   42101003702                     C   
172         8123    8123001005                     C   
173        12086   12086016900                     C   
174        26161   26161407600                     C   

    derived_loan_product_type             derived_dwelling_category  \
170            FHA:First Lien  Single Family (1-4 Units):Site-Built   
171            FHA:First Lien  Single Family (1-4 Units):Site-Built   
172   Conventional:Firs

     activity_year                   lei  derived_msa_md state_code  \
190           2018  549300HW662MN1WU8550           38060         AZ   
191           2018  549300HW662MN1WU8550           38940         FL   
192           2018  549300HW662MN1WU8550           48300         WA   
193           2018  549300HW662MN1WU8550           38060         AZ   
194           2018  549300HW662MN1WU8550           35614         NY   

     county_code  census_tract conforming_loan_limit  \
190         4013    4013420214                     C   
191        12111   12111380400                     C   
192        53017   53017950700                     C   
193         4013    4013420210                     C   
194        36081   36081137700                     C   

    derived_loan_product_type             derived_dwelling_category  \
190            FHA:First Lien  Single Family (1-4 Units):Site-Built   
191   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
192            FHA:Firs

     activity_year                   lei  derived_msa_md state_code  \
210           2018  549300HW662MN1WU8550           44140         MA   
211           2018  549300HW662MN1WU8550           99999         NE   
212           2018  549300HW662MN1WU8550           29340         LA   
213           2018  549300HW662MN1WU8550           15380         NY   
214           2018  549300HW662MN1WU8550           45540         FL   

     county_code  census_tract conforming_loan_limit  \
210        25013   25013811000                     C   
211        31053   31053963800                     C   
212        22019   22019001000                     C   
213        36029   36029000800                     C   
214        12119   12119911200                     C   

    derived_loan_product_type             derived_dwelling_category  \
210   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
211            FHA:First Lien  Single Family (1-4 Units):Site-Built   
212   Conventional:Firs

KeyboardInterrupt: 

In [2]:
# To split into smaller csvs of 500,000

divisor = 500000

outfileno = 1
outfile = None

with open('./data/2018_public_lar_csv.csv', 'r') as infile:
    for index, row in enumerate(csv.reader(infile)):
        if index % divisor == 0:
            if outfile is not None:
                outfile.close()
            outfilename = './data/public_lar_2018-{}.csv'.format(outfileno)
            outfile = open(outfilename, 'w')
            outfileno += 1
            writer = csv.writer(outfile)
        writer.writerow(row)

In [27]:
path = './data/public_lar/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame = pd.concat(li, axis=0, ignore_index=True, sort=True)

In [ ]:
mo = ['MO']
stl = ['41180']
stl_c = ['189']
stl_city = ['510']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499999 entries, 0 to 499998
Data columns (total 99 columns):
activity_year                               499999 non-null int64
lei                                         499999 non-null object
derived_msa_md                              499999 non-null int64
state_code                                  498500 non-null object
county_code                                 492631 non-null float64
census_tract                                492409 non-null float64
conforming_loan_limit                       498625 non-null object
derived_loan_product_type                   499999 non-null object
derived_dwelling_category                   499999 non-null object
derived_ethnicity                           499999 non-null object
derived_race                                499999 non-null object
derived_sex                                 499999 non-null object
action_taken                                499999 non-null int64
purchaser_type     

In [9]:
d1_mo = d1.loc[d1['state_code'].isin(mo)]

In [20]:
frames.append(d1_mo)

In [21]:
print frames

[        activity_year                   lei  derived_msa_md state_code  \
154              2018  549300HW662MN1WU8550           28140         MO   
197              2018  549300HW662MN1WU8550           41180         MO   
347              2018  549300HW662MN1WU8550           28140         MO   
394              2018  549300HW662MN1WU8550           41180         MO   
399              2018  549300HW662MN1WU8550           41180         MO   
479              2018  549300HW662MN1WU8550           41180         MO   
946              2018  549300HW662MN1WU8550           41180         MO   
1304             2018  549300HW662MN1WU8550           41180         MO   
1335             2018  549300HW662MN1WU8550           41180         MO   
1466             2018  549300HW662MN1WU8550           41180         MO   
1627             2018  549300HW662MN1WU8550           41180         MO   
1903             2018  549300HW662MN1WU8550           41180         MO   
1933             2018  549300HW662MN1

In [15]:
d1_stl = d1.loc[d1['derived_msa_md'].isin(stl)]

In [16]:
d1_stl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5311 entries, 197 to 499762
Data columns (total 99 columns):
activity_year                               5311 non-null int64
lei                                         5311 non-null object
derived_msa_md                              5311 non-null int64
state_code                                  5311 non-null object
county_code                                 5311 non-null float64
census_tract                                5311 non-null float64
conforming_loan_limit                       5291 non-null object
derived_loan_product_type                   5311 non-null object
derived_dwelling_category                   5311 non-null object
derived_ethnicity                           5311 non-null object
derived_race                                5311 non-null object
derived_sex                                 5311 non-null object
action_taken                                5311 non-null int64
purchaser_type                              5

In [ ]:
d1_stl = d1.loc[d1['derived_msa_md'].isin(stl)]